<h1 align=center><font size = 8>Data Science Capstone Project</font></h1>
<h1 align=center><font size = 8>A Comparison of Dallas and Fort Worth</font></h1>

## Introduction
<blockquote>It is often said that Dallas is where the east ends and Fort Worth is where the west begins.  Certainly, it is true that when you think of Fort Worth, you think of pickup trucks and cowboy boots and Billy Bob's at The Stockyards and when you think of Dallas, you think of BMW's and banking and Highland Park.  The question for someone who's company has recently moved to the DFW area, though, is:  is there a difference in the neighborhoods in Fort Worth and Dallas?</blockquote>
<blockquote>This project will use Zillow nighborhood data to see if there are any systematic differences in Dallas vs. Fort Worth nighborhoods using k-means clustering.</blockquote>

We will start by loading the libraries that we need for this project.

In [1]:
try:
    import pandas as pd
    import numpy as np
except:
    !conda install -c anaconda pandas numpy --yes 
    import pandas as pd
    import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
try:
    from geopy.geocoders import Nominatim # to convert address into latitude and longitude
except:
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim
try:
    import folium # plotting library
except:
    !conda install -c conda-forge folium --yes 
    import folium
from bs4 import BeautifulSoup
import urllib.request
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import geojson

## Data
The geodata used for this project is the Zillow US Neighborhoods dataset.  Zillow makes this data available for free under a creative commons licence.  It can be downloaded at "https://data.opendatasoft.com/explore/dataset/zillow-neighborhoods%40public/export/".
<br>
<br>
We used the export filters and downloaded one goejson file for Fort Worth and a second geojson file for Dallas.  The files contain full geoshape data and hav "properties" sections that that contain the city, neighborhood name, regionid, geo_point_2d, country and state.  We parse the files to just load the properties section and append it to a dataframe.  We then split the geo_data_2d entry into Latitude and Longitude columns, dropping any rows where geo_data_2d is NA.


In [2]:
with open('e:/downloads/zillow-neighborhoods-FortWorth.geojson') as json_data:
    fortworth_neighborhood_data = geojson.load(json_data)
with open('e:/downloads/zillow-neighborhoods-Dallas.geojson') as json_data:
    dallas_neighborhood_data = geojson.load(json_data)
print(type(fortworth_neighborhood_data), type(dallas_neighborhood_data))

<class 'geojson.feature.FeatureCollection'> <class 'geojson.feature.FeatureCollection'>


In [4]:
fortworth_neighborhood_data = fortworth_neighborhood_data['features']
df = pd.DataFrame(columns=['city','name','regionid','geo_point_2d','county','state'])
for entry in fortworth_neighborhood_data:
    df = df.append( entry['properties'],ignore_index=True)
dallas_neighborhood_data = dallas_neighborhood_data['features']
for entry in dallas_neighborhood_data:
    df = df.append( entry['properties'],ignore_index=True)
df = df.rename(columns={'name': 'Neighborhood'})
df.head()

,city,Neighborhood,regionid,geo_point_2d,county,state
0,Fort Worth,Briercliff,422763,"[32.64714146486756, -97.43035652827258]",Tarrant,TX
1,Fort Worth,Fairmount,233172,"[32.724372978687654, -97.33766189387364]",Tarrant,TX
2,Fort Worth,Willow Creek,207722,"[32.63110091743264, -97.3434603767844]",Tarrant,TX
3,Fort Worth,Shaw Clarke,422871,"[32.70111450603923, -97.33719612060821]",Tarrant,TX
4,Fort Worth,Carver Heights,422820,"[32.72505076405124, -97.22932078449055]",Tarrant,TX


Split up geo_point_2d into Latitude and Longitude

In [5]:
print(df.shape)
df = df.dropna(subset=['geo_point_2d'])
print(df.shape)
df2 = pd.DataFrame(df["geo_point_2d"].tolist(), columns=['Latitude','Longitude']) #pd.DataFrame(df['geo_point_2d'].tolist(),index=df.index)
df = pd.concat([df, df2], axis=1)
df.drop(['geo_point_2d','regionid'],axis=1,inplace=True)
df.head()

(425, 6)
(425, 6)


,city,Neighborhood,county,state,Latitude,Longitude
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321


Extract Dallas-Fort Worth

In [8]:
#dfw = df[df['city'].isin(['Dallas', 'Fort Worth'])]
dfw = df
print(dfw.shape)
dfw

(425, 6)


,city,Neighborhood,county,state,Latitude,Longitude
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321
5,Fort Worth,Santa Fe Enclave,Tarrant,TX,32.868511,-97.319341
6,Fort Worth,Park Place,Tarrant,TX,32.879230,-97.281318
7,Fort Worth,Harbour Points,Tarrant,TX,32.892593,-97.452058
8,Fort Worth,Alexandra Meadows,Tarrant,TX,32.857766,-97.329245
9,Fort Worth,Chapel Creek,Tarrant,TX,32.735927,-97.504439


# Methodology
##  Visual Inspection:  Create a map of DFW Neighborhoods

In [7]:
# create map - use Arlington for map lat/long
address = 'Arlington, TX'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Arlington are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Arlington are 32.701938999999996, -97.10562379033699.


In [10]:
# create map of Toronto using latitude and longitude values
map_dfw = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighborhood in zip(dfw['Latitude'], dfw['Longitude'], dfw['city'], df['Neighborhood']):
    label = '{}, {}, {}, {}'.format(neighborhood, city, lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dfw)  
    
map_dfw

# Get Neighborhood Venues

In [239]:
CLIENT_ID = 'YPF4NY2VJMYEMROHWYBYVI11IZKQ0H4EDECKGJR0XDYRWE4M' # your Foursquare ID
CLIENT_SECRET = 'LI202HN0RHIHWIIQQPWSYIPUQE14DLGSNALI21B5XGV4E31Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1200 #1600 approx 1 mile, 500 approx .3 mile

In [240]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:    
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('\tProblem getting info')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [241]:
dfw_venues = getNearbyVenues(names=dfw['Neighborhood'],
                                   latitudes=dfw['Latitude'],
                                   longitudes=dfw['Longitude']
                                  )
print('There are {} uniques categories.'.format(len(dfw_venues['Venue Category'].unique())))

Briercliff
Fairmount
Willow Creek
Shaw Clarke
Carver Heights
Santa Fe Enclave
Park Place
Harbour Points
Alexandra Meadows
Chapel Creek
Stop Six
Mitchell Boulevard
Jennings - May St. Louis
Hollow Hills
Arlington Heights
Fossil Hill Estates
Overton South
Oakridge Terrace
Scenic Bluff
Sendera Ranch
Diamond Hill - Jarvis
Hamlet
Basswood Park
Sunset Heights South
The Crossing of Fossil Creek
Sylvan Heights West
South Edgewood
Panther Heights
Hulen Heights
Marine Park
Vista Ridge
Basswood Village
Paschal
Lake Country
Worth Heights
Burton Hill Trinity Trails
Stone Meadow
Villages of Woodland Springs
Crestline Area
Candle Ridge West
Caville
Central Chapel Creek
Westcliff
The Trails of Marine Creek
Sunset Hills
Far East
Park Hill
Glencrest
Emory Place
Alamo Heights
Ridglea Hills
South Meadows Crossing
Chadwick Farms
El Poly Pyramid
Eagle Ranch
Poynter Crossing
Westpoint
Deer Creek North
Bonnie Brae
Bently Village
Brittany Place
Bomber Heights
Northbrook
Western Hills North
Scenic Shores
The Vin

In [242]:
dfw_venues.loc[dfw_venues['Venue Category']=='Zoo Exhibit']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
154,Park Hill,32.718465,-97.358118,Penguins,32.719308,-97.354746,Zoo Exhibit
155,Park Hill,32.718465,-97.358118,The Texas Wild,32.717984,-97.353705,Zoo Exhibit
156,Park Hill,32.718465,-97.358118,Parrot Paradise,32.721081,-97.356824,Zoo Exhibit
159,Park Hill,32.718465,-97.358118,World of Primates,32.722699,-97.357197,Zoo Exhibit
160,Park Hill,32.718465,-97.358118,African Savannah,32.722285,-97.358079,Zoo Exhibit
161,Park Hill,32.718465,-97.358118,Asian Falls,32.721702,-97.358469,Zoo Exhibit
162,Park Hill,32.718465,-97.358118,Australian Outback,32.719960,-97.355498,Zoo Exhibit


In [243]:
venue_groups = pd.read_csv('VenueGroups.csv')
venue_groups.head()
dfw_venues = dfw_venues.join(venue_groups.set_index('Venue Category'), on='Venue Category')
dfw_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,Fairmount,32.724373,-97.337662,Old Home Supply,32.724536,-97.335056,Furniture / Home Store,Home Shop
1,Fairmount,32.724373,-97.337662,Rosen House Inn,32.725523,-97.338108,Bed & Breakfast,Lodging
2,Fairmount,32.724373,-97.337662,Dreamy Life Records,32.726719,-97.339484,Record Shop,Music Store
3,Fairmount,32.724373,-97.337662,Krystal's Liquor,32.721594,-97.333647,Liquor Store,Liquor Store
4,Willow Creek,32.631101,-97.343460,Family Dollar,32.634891,-97.345616,Discount Store,Shopping Store


In [244]:
dfw_venues[dfw_venues['Venue Group']=='']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group


In [270]:
print(dfw_venues.groupby('Neighborhood').count())

Neighborhood Latitude  \
Neighborhood                                               
Alamo Heights                                          5   
Alexandra Meadows                                      2   
Almeta,Bonita, Bella Vista                             3   
Altemesa East                                          2   
Apollo Arapaho & Camelot                               3   
Arapaho                                                2   
Arcadia Park                                           6   
Arlington Heights                                      5   
Arts District                                         34   
Avondale                                               6   
Bal Harbour                                            4   
Basswood Park                                         12   
Basswood Village                                       3   
Bear Creek                                             3   
Beechwood Creek                                        2   
Bel Air of Jose

# Analyze Neighborhoods

In [246]:
# one hot encoding
dfw_onehot = pd.get_dummies(dfw_venues[['Venue Category']], prefix="", prefix_sep="")
# There is Neighborhood in the venue categories, so we drop that column
try:
    del dfw_onehot['Neighborhood']
except:
    print('No venues called Neighborhood')
# add neighborhood column back to dataframe
dfw_onehot['Neighborhood'] = dfw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfw_onehot.columns[-1]] + list(dfw_onehot.columns[:-1])
dfw_onehot = dfw_onehot[fixed_columns]
dfw_onehot.head()

No venues called Neighborhood


,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Stadium,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Fairmount,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [247]:
dfw_grouped = dfw_onehot.groupby('Neighborhood').mean().reset_index()
dfw_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Stadium,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Service,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Alamo Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000

# Group most common venues in each neighborhood

In [248]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
top_venue_columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        top_venue_columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        top_venue_columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=top_venue_columns)
neighborhoods_venues_sorted['Neighborhood'] = dfw_grouped['Neighborhood']

for ind in np.arange(dfw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Heights,Thai Restaurant,Taco Place,Thrift / Vintage Store,Diner,Sandwich Place,Ethiopian Restaurant,Dry Cleaner,Egyptian Restaurant,Electronics Store,Event Service
1,Alexandra Meadows,Park,Pool,Zoo Exhibit,Event Service,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Factory
2,"Almeta,Bonita, Bella Vista",Construction & Landscaping,Home Service,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Donut Shop,Factory
3,Altemesa East,Donut Shop,Discount Store,Zoo Exhibit,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
4,Apollo Arapaho & Camelot,Business Service,Spa,Park,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Zoo Exhibit


In [249]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

dfw_grouped_clustering = dfw_grouped.drop('Neighborhood', axis=1)
if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted.drop('Cluster Labels',axis=1, inplace=True)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfw_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dfw_merged = dfw

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfw_merged = dfw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dfw_merged.head() # check the last columns!

,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662,5.0,Bed & Breakfast,Furniture / Home Store,Liquor Store,Record Shop,Factory,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Zoo Exhibit
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460,5.0,Fried Chicken Joint,Discount Store,Zoo Exhibit,Donut Shop,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196,5.0,Playground,Zoo Exhibit,Dog Run,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321,4.0,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


## identify neighborhoods with invalid cluster labels

In [250]:
idx = pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').isna()
print('Number of neighborhoods with invalid Cluster Labels: ',len(dfw_merged[idx]), ' out of ',len(dfw_merged.index))
#neighborhoods_venues_sorted[idx]
dfw_merged[idx]

Number of neighborhoods with invalid Cluster Labels:  55  out of  425


,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Fort Worth,Oakridge Terrace,Tarrant,TX,32.676966,-97.303970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Fort Worth,Sunset Heights South,Tarrant,TX,32.721193,-97.391103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Fort Worth,South Edgewood,Tarrant,TX,32.710697,-97.254413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Fort Worth,Panther Heights,Tarrant,TX,32.595238,-97.408979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Fort Worth,Vista Ridge,Tarrant,TX,32.649997,-97.431163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Fort Worth,Lake Country,Tarrant,TX,32.883233,-97.438576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Fort Worth,Glencrest,Tarrant,TX,32.696808,-97.291868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Fort Worth,Bonnie Brae,Tarrant,TX,32.802555,-97.299576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Fort Worth,Bently Village,Tarrant,TX,32.769712,-97.161811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Map clusters--removing rows with invalid cluster labels and converting cluster labels to int before we map them

In [251]:

# first, we have to drop non-numeric cluster labels and change them to int
dfw_merged = dfw_merged[pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').notnull()]
dfw_merged['Cluster Labels'] = dfw_merged['Cluster Labels'].astype('int')
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow2=['#0000ff','#800080','#ff964f','#ff0000','#008000','#ffd700','#1996f3','#ff7373','#420420','#daa520'] 
rainbow_text=['blue','purple','orange','red','green','yellow','sky blue','salmon','brown?','gold']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfw_merged['Latitude'], dfw_merged['Longitude'], dfw_merged['Neighborhood'], dfw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow2[cluster],
        fill=True,
        fill_color=rainbow2[cluster],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters.save('map1.html') 
map_clusters

## Examine cluster composition

In [252]:
dfw_merged.groupby('Cluster Labels').count()

,city,Neighborhood,county,state,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
2,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
3,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
4,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
5,283,283,283,283,283,283,283,283,283,283,283,283,283,283,283,283


<h3>Original cluster categories</h3>
<ol>Cluster 0 (blue) tends to have Parks and Zoo Exhibits</ol>
<ol>Cluster 1 (purple) this largest category reflects that most of the neighborhoods have a little of everything.</ol>
<ol>Cluster 2 (orange) Gym, Zoo Exhibit, Dry Cleaner, Electronic Store, Egyptian Restaurant and Ethiopian Restaurant, Event Service and Dog Runs possibly reflecting the large number of Ethiopian refugees that came to Dallas-Fort Worth in the late 70's through the 90's and an influx of Coptic Christians that fled Egypt after the removal of Mubarak in 2011.</ol>
<ol>Cluster 3 (red) is about business:  Home Service, Flower Shop, Flea Market, Finacial or Legal Service, Field, Fast Food Restaurant, Farmers Market, Farm Factory, Event Space</ol>
<ol>Cluster 4 (green) is close to sports venues:  Baseball Fields, Football Stadium and Zoo Exhibits and the ubiquitous present of Egyptian and Ethiopian restaurants.</ol>
<ol>Cluster 5 (yellow) is similar to cluster 2, but has Playgrounds and Zoo Exhibits instead of Gyms and lots of Event Spaces (along with Event Services) surrounded by Donut Shops and Dry Cleaners along with Ethiopian and Egyptian Restaurants. (Playground, Zoo Exhibit, Event Space, Donut Shop, Dry Cleaner, Egyptian Restaurant, Electronics Store, Ethiopian Restaurant, Even Service, Factory)</ol>

In [253]:
dfw_venues.loc[dfw_venues['Venue Category']=='Moving Target']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
88,Sylvan Heights West,32.787450,-97.305139,G.L. Hunt Foundation Repair,32.784357,-97.308415,Moving Target,Moving Target
145,Sunset Hills,32.921273,-97.303009,Garage Del Junk,32.919088,-97.306043,Moving Target,Moving Target
197,Northbrook,32.853894,-97.324817,Between Here&There,32.852687,-97.326280,Moving Target,Moving Target
1103,Forest Crest,32.914004,-96.685487,Favor Delivery,32.916014,-96.683446,Moving Target,Moving Target
1208,Whitlock Warriors,32.964145,-96.909702,Ryan's Moving Co.,32.967258,-96.911041,Moving Target,Moving Target
1851,Greens Estates,32.980105,-96.620426,Campbell Rd,32.978813,-96.625488,Moving Target,Moving Target


In [254]:
dfw_merged.loc[dfw_merged['Neighborhood']=='Wolf Creek']

,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
368,Dallas,Wolf Creek,Dallas,TX,32.6584,-96.846397,4,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


In [255]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 0, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
410,Carrollton,Woodcreek,0,Smoke Shop,Furniture / Home Store,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
97,Fort Worth,Waterview at Enchanted Bay,0,Home Service,Furniture / Home Store,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
372,Garland,Dalcastle & Linda Heights,0,Smoke Shop,Astrologer,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm,Donut Shop
407,Garland,Valley Creek,0,Furniture / Home Store,Zoo Exhibit,Flower Shop,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space


In [256]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 1, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
301,Dallas,Eagle Ford,1,Convenience Store,Mexican Restaurant,Thrift / Vintage Store,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
251,Dallas,Winnetka Heights,1,Convenience Store,Mexican Restaurant,Discount Store,Grocery Store,Pizza Place,Taco Place,Liquor Store,Event Service,Egyptian Restaurant,Electronics Store
14,Fort Worth,Arlington Heights,1,Community Center,Park,Ethiopian Restaurant,Concert Hall,Convenience Store,Flea Market,Field,Floating Market,Fast Food Restaurant,Dry Cleaner
31,Fort Worth,Basswood Village,1,Construction & Landscaping,Home Service,Convenience Store,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market
61,Fort Worth,Bomber Heights,1,Convenience Store,Martial Arts Dojo,Zoo Exhibit,Dry Cleaner,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
164,Fort Worth,Bunch-Ellington,1,Convenience Store,Pawn Shop,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market
40,Fort Worth,Caville,1,Convenience Store,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
92,Fort Worth,Eastland,1,Convenience Store,Discount Store,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
53,Fort Worth,El Poly Pyramid,1,Community Center,Convenience Store,Mexican Restaurant,Dry Cleaner,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
212,Fort Worth,Heritage,1,Pet Store,Convenience Store,Grocery Store,Gym,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Dog Run


In [257]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 2, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,Desoto,Frost farms,2,Home Service,Athletics & Sports,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
85,Fort Worth,Carter Park,2,Home Service,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
48,Fort Worth,Emory Place,2,Home Service,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
185,Fort Worth,Hampton Place,2,Home Service,Insurance Office,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
204,Fort Worth,Handley,2,Home Service,Pub,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory
114,Fort Worth,Highland Hills,2,Home Service,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
28,Fort Worth,Hulen Heights,2,Scenic Lookout,Home Service,Bar,Zoo Exhibit,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory
208,Fort Worth,Meadow Creek,2,Home Service,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
6,Fort Worth,Park Place,2,Home Service,Garden Center,Bus Station,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory
55,Fort Worth,Poynter Crossing,2,Home Service,Pool,Food,Playground,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service


In [258]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 3, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
307,Dallas,Southwest Dallas,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
139,Fort Worth,Central Meadowbrook,3,Food,Golf Course,Dog Run,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
217,Fort Worth,East Lake Worth,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
73,Fort Worth,Hillside Morningside,3,Food,Basketball Court,Zoo Exhibit,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
84,Fort Worth,Silver Ridge,3,Food,Beer Bar,Zoo Exhibit,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
284,Garland,Brentwood Place,3,Food,Playground,Dog Run,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
266,Garland,Firewheel Estates,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
329,Garland,Meadowood,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
389,Garland,Oaks of Stoney Creek,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
356,Irving,Belt Line,3,Food,Historic Site,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space


In [259]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 4, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
386,Carrollton,Parks of Carrollton,4,Mexican Restaurant,Park,Sandwich Place,Zoo Exhibit,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
376,Dallas,Lake Highlands,4,Park,Doctor's Office,Business Service,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory
283,Dallas,South Boulevard-Park Row Historic,4,Park,Liquor Store,Zoo Exhibit,Donut Shop,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
368,Dallas,Wolf Creek,4,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
8,Fort Worth,Alexandra Meadows,4,Park,Pool,Zoo Exhibit,Event Service,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Factory
126,Fort Worth,Candleridge,4,Park,Lake,Zoo Exhibit,Donut Shop,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
4,Fort Worth,Carver Heights,4,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
101,Fort Worth,Chisholm Ridge,4,Fast Food Restaurant,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Farmers Market,Farm,Factory
156,Fort Worth,Crestwood,4,Baseball Field,Park,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
54,Fort Worth,Eagle Ranch,4,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


In [260]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 5, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
274,Carrollton,Bel Air of Josey Ranch,5,Home Service,Business Service,Pizza Place,Chinese Restaurant,Zoo Exhibit,Event Service,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
313,Carrollton,Cambridge Estates,5,Korean Restaurant,Hawaiian Restaurant,Park,Japanese Restaurant,Noodle House,Bakery,Dessert Shop,Home Service,Farmers Market,Farm
312,Carrollton,Carrollton Heights,5,History Museum,Furniture / Home Store,Insurance Office,Optical Shop,Donut Shop,Event Service,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
322,Carrollton,Carrollton Summertree,5,Home Service,Dry Cleaner,Park,Playground,Trail,Zoo Exhibit,Event Service,Donut Shop,Egyptian Restaurant,Electronics Store
388,Carrollton,Carrolton Highlands,5,Recreation Center,Optical Shop,Zoo Exhibit,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Event Service,Doctor's Office
349,Carrollton,Hill'n Dale,5,Baseball Field,BBQ Joint,Trail,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Zoo Exhibit,Dry Cleaner
408,Carrollton,Jackson Arms,5,Sandwich Place,Pizza Place,Convenience Store,Shipping Store,Juice Bar,Ice Cream Shop,Salon / Barbershop,Mexican Restaurant,Bank,Fast Food Restaurant
323,Carrollton,Morningside,5,Smoke Shop,Wine Shop,Latin American Restaurant,Gas Station,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
415,Carrollton,Nob Hill,5,Restaurant,Coffee Shop,Gym,Trail,Ethiopian Restaurant,Dog Run,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store
391,Carrollton,Oak Tree North,5,Smoothie Shop,American Restaurant,Café,Shopping Mall,Gym / Fitness Center,Automotive Shop,Martial Arts Dojo,Cosmetics Shop,Flower Shop,Floating Market


## Analyze Neighborhoods by Venue Group

In [261]:
# one hot encoding
dfw_onehot = pd.get_dummies(dfw_venues[['Venue Group']], prefix="", prefix_sep="")
# There is Neighborhood in the venue categories, so we drop that column
try:
    del dfw_onehot['Neighborhood']
except:
    print('No venues called Neighborhood')
# add neighborhood column back to dataframe
dfw_onehot['Neighborhood'] = dfw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfw_onehot.columns[-1]] + list(dfw_onehot.columns[:-1])
dfw_onehot = dfw_onehot[fixed_columns]
dfw_grouped2 = dfw_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods_venues_sorted = pd.DataFrame(columns=top_venue_columns)
neighborhoods_venues_sorted['Neighborhood'] = dfw_grouped2['Neighborhood']

for ind in np.arange(dfw_grouped2.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfw_grouped2.iloc[ind, :], num_top_venues)
# cluster analysis
dfw_grouped_clustering = dfw_grouped2.drop('Neighborhood', axis=1)
if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted.drop('Cluster Labels',axis=1, inplace=True)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfw_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dfw_merged = dfw

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfw_merged = dfw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dfw_merged.head() # check the last columns!
idx = pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').isna()
print('Number of neighborhoods with invalid Cluster Labels: ',len(dfw_merged[idx]), ' out of ',len(dfw_merged.index))
print(neighborhoods_venues_sorted[idx])



No venues called Neighborhood
Number of neighborhoods with invalid Cluster Labels:  55  out of  425
     Cluster Labels                       Neighborhood  \
0                 0                      Alamo Heights   
17                0                Bellaire Park North   
23                2                     Bomber Heights   
26                0                  Brentwood Village   
27                3                Brentwood-Oak Hills   
30                3                   Broadway Village   
33                2                    Buckner Terrace   
47                2                Carrolton Highlands   
58                0                   Chandler Heights   
59                0                       Chapel Creek   
75                0                          Creekside   
78                2                          Crestwood   
81                0                       Crystal Lake   
89                3                           Downtown   
90                0           

In [271]:
dfw_grouped2.head()

,Neighborhood,Adult Boutique,African Cuisine,Airport-Related,American Cuisine,Art/Antiques,Arts & Crafts Store,Arts/Entertainment,Asian Cuisine,Astrologer,Athletic Store,Athletic Venue,Automotive Venue,Boat or Ferry,Bookstore,Breakfast Spot,Bridge,Bubble Tea Shop,Business Service,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Chocolate Shop,Comic Shop,Community Center,Concert Hall,Convenience Store,Dance Studio,Design Studio,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory,Farm,Farmers Market,Fast-Food Venue,Field,Financial Services,Flea Market,Floating Market,Flower Shop,French Cuisine,Gaming Cafe,Garden,Garden Center,Gas Station,Grocery Store,Gun Shop,Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Health Services,Historic Site,Home Service,Home Shop,Hookah Bar,IT Services,Indoor Play Area,Internet Cafe,Intersection,Italian Cuisine,Jewelry Store,Light Rail Station,Liquor Store,Lodging,Massage Studio,Mediterranean Cuisine,Mexican Cuisine,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moving Target,Museum,Music Store,Optical Shop,Outdoor Destination,Outdoor Store,Paper / Office Supplies Store,Park,Pawn Shop,Personal Services,Pet Service,Photography Studio,Playground,Plaza,Professional Services,Recording Studio,Rental Car Location,Rental Service,River,Road,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Store,Smoke Shop,South American Cuisine,Southern / Soul Food Restaurant,Spa,Supplement Shop,Tea Room,Trail,Transportation Venue,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Weight Loss Center,Yoga Studio,Zoo Exhibit
0,Alamo Heights,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alexandra Meadows,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Almeta,Bonita, Bella Vista",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Altemesa East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Apollo Arapaho & Camelot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000

In [262]:
# Create the map
# first, we have to drop non-numeric cluster labels and change them to int
dfw_merged = dfw_merged[pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').notnull()]
dfw_merged['Cluster Labels'] = dfw_merged['Cluster Labels'].astype('int')
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfw_merged['Latitude'], dfw_merged['Longitude'], dfw_merged['Neighborhood'], dfw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow2[cluster],
        fill=True,
        fill_color=rainbow2[cluster],
        fill_opacity=0.7).add_to(map_clusters2)
map_clusters2.save('map2.html')      
map_clusters2

## Examine cluster composition

In [263]:
dfw_merged.groupby('Cluster Labels').count()

,city,Neighborhood,county,state,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,178,178,178,178,178,178,178,178,178,178,178,178,178,178,178,178
1,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
2,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
3,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87
4,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
5,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17


### New cluster categories
<ol>Cluster 0 (blue) tends to have Athletic Venues and Zoo Exhibits</ol>
<ol>Cluster 1 (purple) a lot of Fast Food</ol>
<ol>Cluster 2 (orange) Home Service</ol>
<ol>Cluster 3 (red) Parks</ol>
<ol>Cluster 4 (green) mutt</ol>
<ol>Cluster 5 (yellow) Gyms</ol>

In [264]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 0, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
313,Carrollton,Cambridge Estates,0,Asian Cuisine,Fast-Food Venue,Hawaiian Restaurant,Home Service,Park,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue
312,Carrollton,Carrollton Heights,0,Financial Services,Optical Shop,Home Shop,Museum,Fast-Food Venue,Zoo Exhibit,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
322,Carrollton,Carrollton Summertree,0,Park,Home Service,Dry Cleaner,Trail,Playground,Fast-Food Venue,Design Studio,Dog Run,Drinking Establishment,Electronics Store
323,Carrollton,Morningside,0,Liquor Store,Smoke Shop,South American Cuisine,Gas Station,Factory,Field,Fast-Food Venue,Farmers Market,Farm,Zoo Exhibit
391,Carrollton,Oak Tree North,0,Fast-Food Venue,American Cuisine,Gym,Shopping Store,Automotive Venue,Athletic Venue,Zoo Exhibit,Farm,Field,Farmers Market
270,Carrollton,Park Terrace,0,Outdoor Destination,Home Service,Entertainment Venue,Park,Zoo Exhibit,Field,Drinking Establishment,Dry Cleaner,Electronics Store,Event Service
401,Carrollton,Parkside Estates,0,Playground,Park,Trail,Field,Design Studio,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue
298,Carrollton,Savoy of Josey Ranch,0,Mexican Cuisine,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
317,Carrollton,Whitlock Warriors,0,Outdoor Destination,Pet Service,Moving Target,Business Service,Bookstore,Zoo Exhibit,Field,Dry Cleaner,Electronics Store,Entertainment Venue
410,Carrollton,Woodcreek,0,Home Shop,Smoke Shop,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space


In [265]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 1, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
307,Dallas,Southwest Dallas,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
217,Fort Worth,East Lake Worth,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
84,Fort Worth,Silver Ridge,1,Drinking Establishment,Grocery Store,Zoo Exhibit,Financial Services,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
25,Fort Worth,Sylvan Heights West,1,Grocery Store,Shopping Store,Moving Target,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
284,Garland,Brentwood Place,1,Playground,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
266,Garland,Firewheel Estates,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
329,Garland,Meadowood,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
389,Garland,Oaks of Stoney Creek,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
356,Irving,Belt Line,1,Grocery Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory


In [266]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 2, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
388,Carrollton,Carrolton Highlands,2,Optical Shop,Athletic Venue,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
349,Carrollton,Hill'n Dale,2,Athletic Venue,Trail,Fast-Food Venue,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
325,Dallas,Buckner Terrace,2,American Cuisine,Gym,Athletic Venue,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
8,Fort Worth,Alexandra Meadows,2,Athletic Venue,Park,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
161,Fort Worth,Arcadia Park,2,Athletic Venue,Garden Center,Park,Fast-Food Venue,Arts/Entertainment,Zoo Exhibit,Farmers Market,Farm,Factory,Event Space
77,Fort Worth,Beechwood Creek,2,Playground,Athletic Venue,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
61,Fort Worth,Bomber Heights,2,Convenience Store,Athletic Venue,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
120,Fort Worth,Brookside,2,Athletic Venue,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
139,Fort Worth,Central Meadowbrook,2,Grocery Store,Athletic Venue,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
193,Fort Worth,Colonial Hills,2,Athletic Venue,Gym,Plaza,Spa,Zoo Exhibit,Field,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue


In [267]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 3, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
274,Carrollton,Bel Air of Josey Ranch,3,Home Service,Asian Cuisine,Fast-Food Venue,Business Service,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
408,Carrollton,Jackson Arms,3,Fast-Food Venue,Convenience Store,Shopping Store,Personal Services,Mexican Cuisine,Shipping Store,Health Services,Financial Services,Video Store,Farm
415,Carrollton,Nob Hill,3,American Cuisine,Gym,Trail,Fast-Food Venue,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue
386,Carrollton,Parks of Carrollton,3,Park,Mexican Cuisine,Fast-Food Venue,Field,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
275,Carrollton,Rohton Park,3,Fast-Food Venue,Zoo Exhibit,Design Studio,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
412,Carrollton,Rollingwood Estates,3,Park,Gym,Garden Center,Fast-Food Venue,Field,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue
247,Carrollton,Trinity Mills,3,Fast-Food Venue,Health Services,Home Service,Intersection,Miscellaneous Shop,Mexican Cuisine,Field,Dry Cleaner,Electronics Store,Entertainment Venue
362,Carrollton,Trinity Mills,3,Fast-Food Venue,Health Services,Home Service,Intersection,Miscellaneous Shop,Mexican Cuisine,Field,Dry Cleaner,Electronics Store,Entertainment Venue
256,Dallas,Dells District,3,Fast-Food Venue,Liquor Store,Park,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
285,Dallas,Far North,3,Dance Studio,Fast-Food Venue,Design Studio,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory


In [268]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 4, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,Desoto,Frost farms,4,Home Service,Athletic Venue,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
31,Fort Worth,Basswood Village,4,Home Service,Convenience Store,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
35,Fort Worth,Burton Hill Trinity Trails,4,Home Service,Trail,Field,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service
85,Fort Worth,Carter Park,4,Home Service,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
48,Fort Worth,Emory Place,4,Home Service,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
109,Fort Worth,Falcon Ridge,4,Home Service,Park,Gas Station,Fast-Food Venue,Field,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue
185,Fort Worth,Hampton Place,4,Home Service,Financial Services,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
204,Fort Worth,Handley,4,Drinking Establishment,Home Service,Zoo Exhibit,Flea Market,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
114,Fort Worth,Highland Hills,4,Home Service,Zoo Exhibit,Flea Market,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
13,Fort Worth,Hollow Hills,4,Home Service,Health Food Store,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space


In [269]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 5, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]].sort_values(['city','Neighborhood'])

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
376,Dallas,Lake Highlands,5,Park,Health Services,Business Service,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
283,Dallas,South Boulevard-Park Row Historic,5,Park,Liquor Store,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
368,Dallas,Wolf Creek,5,Park,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
126,Fort Worth,Candleridge,5,Park,Outdoor Destination,Field,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
4,Fort Worth,Carver Heights,5,Park,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
54,Fort Worth,Eagle Ranch,5,Park,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
176,Fort Worth,FW Vista West,5,Park,Electronics Store,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Entertainment Venue,Event Service,Event Space,Factory
165,Fort Worth,Hulen Bend Estates,5,Park,Financial Services,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space,Factory
172,Fort Worth,North Greenbriar,5,Athletic Store,Park,Zoo Exhibit,Financial Services,Drinking Establishment,Dry Cleaner,Electronics Store,Entertainment Venue,Event Service,Event Space
146,Fort Worth,Oakhurst,5,Park,Home Service,South American Cuisine,Fast-Food Venue,Zoo Exhibit,Field,Dog Run,Drinking Establishment,Dry Cleaner,Electronics Store
